In [2]:
import requests
import json
import pandas as pd
import pickle
import re
from sqlalchemy import create_engine
import pymysql

### 아이템 key리스트 받아오기

In [17]:
url = 'http://ddragon.leagueoflegends.com/cdn/12.19.1/data/ko_KR/item.json'
datas = requests.get(url)
datas = datas.json()
keyList = datas['data'].keys()

### 문자열 파싱함수

In [41]:
def rename(st):
    st = st.replace('<br>','\n')
    txt = re.compile('(<([^>]+)>)')
    result = txt.sub('',st)
    result = result[:-1]
    return result

In [49]:
def rename2(st):
    st = st.replace('<br>','\n')
    txt = re.compile('(<([^>]+)>)')
    result = txt.sub('',st)
    return result

In [50]:
item = []
for key in keyList:
    item.append([key,rename2(datas['data'][key]['name']),datas['data'][key]['plaintext'],rename(datas['data'][key]['description'])
         ,datas['data'][key]['gold']['total'],datas['data'][key]['gold']['base'],datas['data'][key]['gold']['sell']
         ,'https://ddragon.leagueoflegends.com/cdn/12.19.1/img/item/'+datas['data'][key]['image']['full']])
df = pd.DataFrame(item, columns=['item_key','name','plaintext','itemDst','total','base','sell','icon'])
df

,item_key,name,plaintext,itemDst,total,base,sell,icon
0,1001,장화,이동 속도가 약간 증가합니다.,이동 속도 25,300,300,210,https://ddragon.leagueoflegends.com/cdn/12.19....
1,1004,요정의 부적,마나 재생량이 약간 증가합니다.,기본 마나 재생 50%,250,250,175,https://ddragon.leagueoflegends.com/cdn/12.19....
2,1006,원기 회복의 구슬,체력 재생량이 약간 증가합니다.,기본 체력 재생 100%,300,300,120,https://ddragon.leagueoflegends.com/cdn/12.19....
3,1011,거인의 허리띠,체력이 대폭 증가합니다.,체력 350,900,500,630,https://ddragon.leagueoflegends.com/cdn/12.19....
4,1018,민첩성의 망토,치명타 확률이 증가합니다.,치명타 확률 15%,600,600,420,https://ddragon.leagueoflegends.com/cdn/12.19....
...,...,...,...,...,...,...,...,...
249,7023,별의 균형,,체력 400\n스킬 가속 25\n방어력 40\n마법 저항력 40\n광휘: 챔피언을 ...,2500,0,1750,https://ddragon.leagueoflegends.com/cdn/12.19....
250,7024,중단,,주문력 90\n체력 350\n마나 800\n스킬 가속 25\n신성한 보호: 보호를 ...,2800,0,1960,https://ddragon.leagueoflegends.com/cdn/12.19....
251,7050,갱플랭크 Placeholder,새로운 갱플랭크 인터페이스 추가 예정!,,0,0,0,https://ddragon.leagueoflegends.com/cdn/12.19....
252,8001,증오의 사슬,,체력 650\n스킬 가속 20\n\n사용 시 - 맹세: 피의 복수를 할 숙적을 선택...,2500,800,1750,https://ddragon.leagueoflegends.com/cdn/12.19....


In [51]:
df.loc[177,'name']

'가차없는 포격\n바다뱀 은화 500닢'

### DB에 저장하기

In [53]:
conn = pymysql.connect(host='3.36.255.55', user='root', password='root1234', db='lol_project', charset='utf8')
engine = create_engine('mysql+pymysql://{user}:{pw}@{host}/{db}'.format(user='root',pw='root1234',db='lol_project',host='3.36.255.55'))
conn = engine.connect()
df.to_sql(name='item', con=engine ,if_exists='append',index=False)
conn.close()

### 피클에 저장하기

In [54]:
with open('itemData.pickle','wb') as f:
    pickle.dump(df,f)